In [ ]:
import time
import copy

In [ ]:
uuid = 0

class parse:
    def __init__(self, raw, offset = 0):
        self.parse(raw, offset)
        self.stack = []
    def parse(self, raw, offset):
        pass
    def append(self, parent):
        pass
    def timeshift(self, offset):
        pass
        
def CInt(raw, offset):
    return offset+4, int.from_bytes(raw[offset:offset+4], "little")
    
def CLong( raw, offset):
    return offset+8, int.from_bytes(raw[offset:offset+8], "little")

def CString( raw, offset):
    result = raw[offset:].decode("ASCII")
    return offset+len(result), result
    
class Alloc( parse ):
    def parse(self, raw, offset):
        offset, self.tid = CLong(raw, offset)
        self.uuid = None
        offset, self.time = CLong(raw, offset)
        offset, self.addr = CLong(raw, offset)
        offset, self.size = CLong(raw, offset)
    def append(self, parent):
        if self.tid in parent["threads"]["active"]:
            self.uuid = parent["threads"]["active"][self.tid]
            parent["threads"]["history"][self.uuid].consumption += self.size
        parent["memory"][self.addr] = self
        parent["last"] = parent["memory"][self.addr]
        parent["consumption"] += self.size
    def timeshift(self, offset):
        self.time -= offset
        
class Free( parse ):
    def parse(self, raw, offset):
        offset, self.tid = CLong(raw, offset)
        offset, self.time = CLong(raw, offset)
        offset, self.addr = CLong(raw, offset)
        offset, self.size = CLong(raw, offset)
        self.uuid = None
    def append(self, parent):
        oldsize = 0
        oldthread = None
        
        if self.addr in parent["memory"]:
            oldsize = parent["memory"][self.addr].size
            oldthread = parent["memory"][self.addr].uuid
            
        if self.addr in parent["memory"]:
        #    parent["memory"][self.addr] = self
            del(parent["memory"][self.addr])
        
        if oldthread:
            if oldthread in parent["threads"]["history"]:
                parent["threads"]["history"][oldthread].consumption -= oldsize  
                
        parent["last"] = None #parent["memory"][self.addr]
        parent["consumption"] -= oldsize
    def timeshift(self, offset):
        self.time -= offset

class Info( parse ):
    def parse(self, raw, offset):
        offset, self.string = CString(raw, offset)
    def append(self, parent):
        print(f"event {self.string}")
        parent["events"].append((parent["counter"], self.string))
        
class Thread( parse ):
    def parse(self, raw, offset):
        global uuid
        offset, self.tid = CLong(raw, offset)
        offset, self.time = CLong(raw, offset)
        offset, self.child_tid = CLong(raw, offset)
        self.uuid = uuid
        uuid += 1
        self.consumption = 0
    def append(self, parent):
        print(f"new thread {self.child_tid} -- {self.uuid}")
        parent["threads"]["active"][self.child_tid] = self.uuid
        parent["threads"]["history"][self.uuid] = self
        parent["last"] = parent["threads"]["history"][self.uuid]
    def timeshift(self, offset):
        self.time -= offset
    
class Stack( parse ):
    def parse(self, raw, offset):
        result = {}
        offset, self.tid = CLong(raw, offset)
        offset, self.time = CLong(raw, offset)
        offset, self.length = CLong(raw, offset)   
    def append(self, parent):
        pass
    def timeshift(self, offset):
        self.time -= offset
    
class Frame( parse ):
    def parse(self, raw, offset):
        offset, self.symbol = CString(raw, offset)
    def append(self, parent):
        parent["last"].stack.append(self.symbol) 
    
actions = { "A": { "type": Alloc },
            "F": { "type": Free },
            "T": { "type": Thread },
            "S": { "type": Stack },
            "s": { "type": Frame },
            "I": { "type": Info }
          }


def evaluate(action, length, data, offset_t):
    item = actions[action]["type"](data)
    item.timeshift(offset_t)
    return item
        
    
def findAndReadLineHeader(log):
    # line must begin with hashtag-action-length tuple, first search for a newline beginning with hashtag
    success = False
    action = None
    length = None
    data = None
    while not success:
        l = b'\0'
        while l != b'#' and len(l) > 0:
            l = log.read(1)
        l = log.read(9)
        action = l[0]
        length = int.from_bytes(l[1:], "little") #8

        # remove newline char
        if length <= 0 or length > 200:
            continue
        data = log.read(length-1)
        newline = log.read(1)
        success = (newline == b"\n")
        if not success:
            print(data, newline)
    return chr(action), length-1, data

def readLine(log, offset_t):
    action, length, data = findAndReadLineHeader(log)
    #print(action, length, data)
    
    item = evaluate(action, length, data, offset_t)
    return item

def filterKeyword(keyword):
    threads = hist["threads"]["history"]
    return sum([ threads[x].consumption / 1024 / 1024 for x in threads if keyword in threads[x].stack[1]])

def calcBlocks():
    addr = [ x for x  in hist["memory"].keys() ]
    addr.sort()
    addr_min = min(addr)
    addr_max = max(addr)
    addr_range = addr_max - addr_min
    result = []
    current = 0
    last = 0
    for a in addr:
        start = a
        length = hist["memory"][a].size
        while length > 0:
            block = int(start/4096)    
            block_free = (block+1) * 4096 - start
            block_length = length if block_free > length else block_free
    
            last_block = 0
            if last:
                last_block = last
    
            if last_block == block:
                result[-1] += block_length
            else:
                result.append(block_length)
    
            length = length - block_length
            start = start + block_length + 1
            last = block    
    return sum(result) / 1024 / 1024, len(result)*4/1024

In [ ]:
log = open("mallog.log", "rb")
hist = {"memory": {}, "threads": {"active": {}, "history": {}}, "last": None, "events": [], "consumption": 0, "counter": 0}
overall = []
threadmem = []
blocks = []

In [ ]:
item = readLine(log, 0)
offset_t = item.time
main_t = item.tid
dummy_data = b'\0'*24
hist["threads"]["active"][main_t] = 0
hist["threads"]["history"][0] = Thread(dummy_data)
item.time = 0

item.append(hist)

interval = 50000
while True:
    try:
        item = readLine(log, offset_t)
        fails = 100
    except:
        fails -= 1
        if fails == 0:
            break
        continue
    
    if hist["counter"] % interval == 0:
        #print()        
        threads = hist["threads"]["history"]
        while len(threadmem) < len(threads):
            threadmem.append([])
        for t in threads:
            threadmem[t].append((hist["counter"], threads[t].consumption/1024/1024))
        overall.append((hist["counter"], hist["consumption"]/1024/1024))
        
        compressed, fragmented = calcBlocks()
        blocks.append((hist["counter"], compressed, fragmented))
        
        if hist["counter"] % ( interval * 100 ) == 0:
            print(f"iter {hist['counter']} -- {overall[-1][1]} MB")
            print(f"  -- compressed {compressed} MB")
            print(f"  -- fragmented {fragmented} MB")
        
    item.append(hist)
    hist["counter"] += 1

threads = hist["threads"]["history"]
while len(threadmem) < len(threads):
    threadmem.append([])
for t in threads:
    threadmem[t].append((hist["counter"], threads[t].consumption/1024/1024))    
overall.append((hist["counter"], hist["consumption"]/1024/1024))
                  
compressed, fragmented = calcBlocks()
blocks.append((hist["counter"], compressed, fragmented))
print()
print(f"iter {hist['counter']} -- {overall[-1][1]} MB")
print(f"  -- compressed {compressed} MB")
print(f"  -- fragmented {fragmented} MB")
print("end")


In [ ]:
import matplotlib.pyplot as plt

plt.plot([x[0] for x in overall], [x[1] for x in overall])
plt.plot([x[0] for x in blocks], [x[1] for x in blocks], label="compressed")
plt.plot([x[0] for x in blocks], [x[2] for x in blocks], label="fragmented")
for x in hist["events"]:
    plt.axvline(x[0], color="red")
plt.show()



In [ ]:
import matplotlib.pyplot as plt
for no in range(len(hist["threads"]["history"])):
    print()
    print(no)
    for s in hist["threads"]["history"][no].stack:
        print(s)
    plt.plot([x[0] for x in threadmem[no]], [x[1] for x in threadmem[no]])
    for x in hist["events"]:
        plt.axvline(x[0], color="red")
    plt.show()